In [65]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.tree import export_graphviz
from sklearn.metrics import r2_score
import warnings
from sklearn.model_selection import cross_val_score, KFold
from sklearn.base import clone  # Import clone here

In [96]:
import sqlite3

In [66]:
from sklearn.metrics import r2_score


In [67]:
df_game = pd.read_csv(r'/Users/eyzhang28/Downloads/archive (2)/csv/game.csv')

In [69]:
df_game['game_year'] = np.nan
df_game['game_year'] = df_game['game_date'].apply(lambda v: v[0:4])

In [70]:
df1 = df_game[df_game['season_type'] == 'Regular Season']
df1 = df1[(df1['game_year'] > '2012') & (df1['game_id'] < 22100463)]

In [71]:
df1['pos_home'] = np.nan
df1['pos_away'] = np.nan

In [72]:
df1['pos_home'] = (df1['fga_home'] - df1['oreb_home']) + df1['tov_home'] + 0.44*(df1['fta_home'])
df1['pos_away'] = (df1['fga_away'] - df1['oreb_away']) + df1['tov_away'] + 0.44*(df1['fta_away'])

In [73]:
def teamname(df, team):
    df['team'] = team
    return df

In [74]:
def calc_ratings(df, team):
    df['offensive_rating'] = np.nan
    df['defensive_rating'] = np.nan
    for i in range(len(df)):
        if df['team_abbreviation_home'].iloc[i] == team:
            df['offensive_rating'].iloc[i] = df['pts_home'].iloc[i]/df['pos_home'].iloc[i]*100
            df['defensive_rating'].iloc[i] = df['pts_away'].iloc[i]/df['pos_away'].iloc[i]*100
        else:
            df['offensive_rating'].iloc[i] = df['pts_away'].iloc[i]/df['pos_away'].iloc[i]*100
            df['defensive_rating'].iloc[i] = df['pts_home'].iloc[i]/df['pos_home'].iloc[i]*100
    return df

In [75]:
def home_game_indic(df, team):
    df['home_game_indicator'] = np.nan
    for i in range(len(df)):
        if df['team_abbreviation_home'].iloc[i] == team:
            df['home_game_indicator'].iloc[i] = 1
        else:
            df['home_game_indicator'].iloc[i] = 0
    return df

In [76]:
def game_results(df, team):
    df['spread'] = np.nan
    for i in range(len(df)):
        if df['team_abbreviation_home'].iloc[i] == team:
            df['spread'].iloc[i] = df['pts_home'].iloc[i] - df['pts_away'].iloc[i]
        else:
            df['spread'].iloc[i] = df['pts_away'].iloc[i] - df['pts_home'].iloc[i]
    return df

In [77]:
def days_rest(df, team):
    df['rest'] = np.nan
    for i in range(len(df)):
        try:
            df['rest'].iloc[i] = (pd.to_datetime(df['game_date'].iloc[i]) - pd.to_datetime(df['game_date'].iloc[i - 1])).days
        except:
            pass
    return df

In [78]:
def calculate_rating_ema(df, window):
    df['offensive_rating_20_day_ma'] = df['offensive_rating'].rolling(window=window, min_periods = 1).mean().shift(1)
    df['offensive_rating_20_day_ema'] = df['offensive_rating'].ewm(span = window, adjust = True).mean().shift(1)
    df['defensive_rating_20_day_ma'] = df['defensive_rating'].rolling(window= window, min_periods = 1).mean().shift(1)
    df['defensive_rating_20_day_ema'] = df['defensive_rating'].ewm(span = window, adjust = True).mean().shift(1)
    
    return df


In [79]:
df1.columns

Index(['season_id', 'team_id_home', 'team_abbreviation_home', 'team_name_home',
       'game_id', 'game_date', 'matchup_home', 'wl_home', 'min', 'fgm_home',
       'fga_home', 'fg_pct_home', 'fg3m_home', 'fg3a_home', 'fg3_pct_home',
       'ftm_home', 'fta_home', 'ft_pct_home', 'oreb_home', 'dreb_home',
       'reb_home', 'ast_home', 'stl_home', 'blk_home', 'tov_home', 'pf_home',
       'pts_home', 'plus_minus_home', 'video_available_home', 'team_id_away',
       'team_abbreviation_away', 'team_name_away', 'matchup_away', 'wl_away',
       'fgm_away', 'fga_away', 'fg_pct_away', 'fg3m_away', 'fg3a_away',
       'fg3_pct_away', 'ftm_away', 'fta_away', 'ft_pct_away', 'oreb_away',
       'dreb_away', 'reb_away', 'ast_away', 'stl_away', 'blk_away', 'tov_away',
       'pf_away', 'pts_away', 'plus_minus_away', 'video_available_away',
       'season_type', 'game_year', 'pos_home', 'pos_away'],
      dtype='object')

In [80]:
df_parsed = pd.DataFrame()
for team in df1['team_abbreviation_home'].unique():
    print(team)
    df_team = df1[(df1['team_abbreviation_home'] == team) | (df1['team_abbreviation_away'] == team)]
    df_team = df_team.reset_index()
    df_team = calc_ratings(df_team, team)
    df_team = home_game_indic(df_team, team)
    df_team = game_results(df_team, team)
    df_team = days_rest(df_team, team)
    df_team = teamname(df_team, team)
    df_team = calculate_rating_ema(df_team, 20)
    df_team = df_team.reset_index()
    df_temp = df_team[['game_id', 'team', 'rest', 'offensive_rating_20_day_ema', 'defensive_rating_20_day_ema']]
    df_parsed = df_parsed.append(df_temp)

LAL


/Users/eyzhang28/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


IND
MIA
MIN
SAS
UTA
DET
SAC
TOR
NYK
DAL
CLE
PHX
PHI
GSW
HOU
NOP
CHI
LAC
ATL
DEN
CHA
MEM
BOS
WAS
BKN
ORL
POR
MIL
OKC


In [106]:
df2 = df1.copy()

In [107]:
df2 = df2[['team_abbreviation_home', 'team_abbreviation_away', 'game_id', 'pts_home', 'pts_away', 'game_year', 'pos_home', 'pos_away']]
df2['spread'] = df2['pts_home'] - df2['pts_away']
df2['rest_home'] = np.nan
df2['rest_away'] = np.nan
df2['OR_home'] = np.nan
df2['OR_away'] = np.nan
df2['DR_home'] = np.nan
df2['DR_away'] = np.nan
df2['rest_diff'] = np.nan

In [108]:
df2 = df2.reset_index(drop = True)

In [109]:
for i in range(len(df2)):
    home_team = df2['team_abbreviation_home'].iloc[i]
    away_team = df2['team_abbreviation_away'].iloc[i]
    game_id = df2['game_id'].iloc[i]
    temp_df = df_parsed[(df_parsed['team'] == home_team) & (df_parsed['game_id'] == game_id)]
    df2['rest_home'].iloc[i] = temp_df['rest'].iloc[0]
    df2['OR_home'].iloc[i] = temp_df['offensive_rating_20_day_ema'].iloc[0]
    df2['DR_home'].iloc[i] = temp_df['defensive_rating_20_day_ema'].iloc[0]
    temp_df = df_parsed[(df_parsed['team'] == away_team) & (df_parsed['game_id'] == game_id)]
    df2['rest_away'].iloc[i] = temp_df['rest'].iloc[0]
    df2['OR_away'].iloc[i] = temp_df['offensive_rating_20_day_ema'].iloc[0]
    df2['DR_away'].iloc[i] = temp_df['defensive_rating_20_day_ema'].iloc[0]
    

/Users/eyzhang28/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [110]:
df2['rest_diff'] = df2['rest_home'] - df2['rest_away']

In [111]:
df2 = df2.dropna()

In [112]:
df2

,team_abbreviation_home,team_abbreviation_away,game_id,pts_home,pts_away,game_year,pos_home,pos_away,spread,rest_home,rest_away,OR_home,OR_away,DR_home,DR_away,rest_diff
17,CHI,NYK,21300018,82.0,81.0,2013,90.36,94.16,1.0,2.0,1.0,93.947785,95.258256,107.257418,84.452584,1.0
18,LAC,GSW,21300019,126.0,115.0,2013,110.00,105.52,11.0,2.0,1.0,103.914447,120.053784,109.104590,88.478916,1.0
19,ATL,TOR,21300023,102.0,95.0,2013,96.64,96.16,7.0,2.0,2.0,105.415861,97.771236,117.342880,92.790102,0.0
20,DEN,POR,21300029,98.0,113.0,2013,98.96,100.60,-15.0,2.0,2.0,90.274928,96.235195,95.622609,109.565950,0.0
21,CHA,CLE,21300020,90.0,84.0,2013,98.96,99.68,6.0,2.0,2.0,87.552743,101.072607,104.849279,95.373377,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9976,PHI,NOP,22100454,117.0,107.0,2022,94.64,91.44,10.0,36.0,39.0,104.660217,106.175901,108.228050,109.521710,-3.0
9977,BKN,DEN,22100453,118.0,124.0,2022,105.44,101.88,-6.0,39.0,40.0,108.131481,111.050090,104.558645,111.161338,-1.0
9978,TOR,CHI,22100430,127.0,120.0,2022,111.36,107.56,7.0,47.0,23.0,108.817953,111.996156,106.856099,106.654585,24.0
9979,TOR,ORL,22100458,97.0,103.0,2022,100.48,103.24,-6.0,29.0,76.0,109.315723,101.117869,107.304626,111.057463,-47.0


In [113]:
db_path = '/Users/eyzhang28/Downloads/basketball-final.sqlite'
conn = sqlite3.connect(db_path)
query = "SELECT * FROM BettingOdds_History;"
df = pd.read_sql_query(query, conn)

In [114]:
df['game_id'] = pd.to_numeric(df['GAME_ID'])
df['spread'] = df['HomeSpread_AtOpen']*-1
df = df[['game_id', 'spread']]

In [115]:
df_eval = pd.merge(df2, df, how = 'left', on = 'game_id')
df_eval = df_eval.dropna()

In [122]:
df_eval_clean = df_eval[(df_eval['spread_y'] < 50) & (df_eval['spread_y'] > -50)]

In [123]:
df_eval

,team_abbreviation_home,team_abbreviation_away,game_id,pts_home,pts_away,game_year,pos_home,pos_away,spread_x,rest_home,rest_away,OR_home,OR_away,DR_home,DR_away,rest_diff,spread_y
0,CHI,NYK,21300018,82.0,81.0,2013,90.36,94.16,1.0,2.0,1.0,93.947785,95.258256,107.257418,84.452584,1.0,7.5
1,LAC,GSW,21300019,126.0,115.0,2013,110.00,105.52,11.0,2.0,1.0,103.914447,120.053784,109.104590,88.478916,1.0,7.0
2,ATL,TOR,21300023,102.0,95.0,2013,96.64,96.16,7.0,2.0,2.0,105.415861,97.771236,117.342880,92.790102,0.0,3.5
3,DEN,POR,21300029,98.0,113.0,2013,98.96,100.60,-15.0,2.0,2.0,90.274928,96.235195,95.622609,109.565950,0.0,6.0
4,CHA,CLE,21300020,90.0,84.0,2013,98.96,99.68,6.0,2.0,2.0,87.552743,101.072607,104.849279,95.373377,0.0,-1.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9953,CHI,HOU,22100459,133.0,118.0,2021,97.56,95.56,15.0,1.0,2.0,107.637888,106.913068,107.547956,113.297093,-1.0,8.5
9954,GSW,SAC,22100462,113.0,98.0,2021,97.96,105.04,15.0,2.0,1.0,108.410390,108.638609,102.431452,111.895453,1.0,13.5
9955,BOS,PHI,22100457,103.0,108.0,2021,100.00,100.64,-5.0,2.0,4.0,109.232439,104.380956,107.585366,108.778371,-2.0,3.0
9956,UTA,CHA,22100461,112.0,102.0,2021,107.28,106.96,10.0,2.0,1.0,117.905315,113.273014,106.258343,117.375505,1.0,11.0


In [124]:
df_eval_clean

,team_abbreviation_home,team_abbreviation_away,game_id,pts_home,pts_away,game_year,pos_home,pos_away,spread_x,rest_home,rest_away,OR_home,OR_away,DR_home,DR_away,rest_diff,spread_y
0,CHI,NYK,21300018,82.0,81.0,2013,90.36,94.16,1.0,2.0,1.0,93.947785,95.258256,107.257418,84.452584,1.0,7.5
1,LAC,GSW,21300019,126.0,115.0,2013,110.00,105.52,11.0,2.0,1.0,103.914447,120.053784,109.104590,88.478916,1.0,7.0
2,ATL,TOR,21300023,102.0,95.0,2013,96.64,96.16,7.0,2.0,2.0,105.415861,97.771236,117.342880,92.790102,0.0,3.5
3,DEN,POR,21300029,98.0,113.0,2013,98.96,100.60,-15.0,2.0,2.0,90.274928,96.235195,95.622609,109.565950,0.0,6.0
4,CHA,CLE,21300020,90.0,84.0,2013,98.96,99.68,6.0,2.0,2.0,87.552743,101.072607,104.849279,95.373377,0.0,-1.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9953,CHI,HOU,22100459,133.0,118.0,2021,97.56,95.56,15.0,1.0,2.0,107.637888,106.913068,107.547956,113.297093,-1.0,8.5
9954,GSW,SAC,22100462,113.0,98.0,2021,97.96,105.04,15.0,2.0,1.0,108.410390,108.638609,102.431452,111.895453,1.0,13.5
9955,BOS,PHI,22100457,103.0,108.0,2021,100.00,100.64,-5.0,2.0,4.0,109.232439,104.380956,107.585366,108.778371,-2.0,3.0
9956,UTA,CHA,22100461,112.0,102.0,2021,107.28,106.96,10.0,2.0,1.0,117.905315,113.273014,106.258343,117.375505,1.0,11.0


In [125]:
df_eval_clean['spread_y'].describe()

count    9955.000000
mean        2.517480
std         6.428041
min       -34.500000
25%        -2.500000
50%         3.000000
75%         7.000000
max        22.000000
Name: spread_y, dtype: float64

In [ ]:
df_eval['spread_y']

In [126]:
pred_col = 'spread_x'
X_train = df_eval_clean[['OR_home', 'OR_away', 'DR_home', 'DR_away', 'rest_diff']]
Y_train = df_eval_clean[pred_col]
Y_eval = df_eval_clean['spread_y']

# Define your pipeline as before
FUT_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('regressor', GradientBoostingRegressor(learning_rate=0.1, max_depth = 3, n_estimators = 300))
])

cv_strategy = KFold(n_splits=5, shuffle=True, random_state=i)

# Prepare a list to store feature importances from each fold
feature_importances = []

# Prepare a list to store the R^2 scores for each fold
r2_scores = []
r2_scores_odds = []

# Loop through each split
for train_index, test_index in cv_strategy.split(X_train):
    # Split the data into training and testing sets for this fold
    X_train_fold, X_test_fold = X_train.iloc[train_index], X_train.iloc[test_index]
    Y_train_fold, Y_test_fold = Y_train.iloc[train_index], Y_train.iloc[test_index]
    Y_eval_fold = Y_eval.iloc[test_index]

    # Clone the pipeline to ensure a fresh model
    clone_pipeline = clone(FUT_pipeline)

    # Fit the pipeline on the training data
    clone_pipeline.fit(X_train_fold, Y_train_fold)

    # Predict on the test set
    Y_pred_fold = clone_pipeline.predict(X_test_fold)

    # Calculate R^2 score and append to list
    r2_scores.append(r2_score(Y_test_fold, Y_pred_fold))
    r2_scores_odds.append(r2_score(Y_test_fold, Y_eval_fold))

    # Access the fitted model from the pipeline and store its feature importances
    feature_importances.append(clone_pipeline.named_steps['regressor'].feature_importances_)

# Calculate the average feature importance across all folds
average_feature_importance = np.mean(feature_importances, axis=0)

# Calculate the mean R^2 score across all folds
mean_r2_score = np.mean(r2_scores)

print(r2_scores)
print(r2_scores_odds)

print("Average Feature Importances:", average_feature_importance)
print("Mean R^2 Score:", mean_r2_score)

[0.10054765759538864, 0.13778331959873535, 0.11823533179774937, 0.12029827029309603, 0.11883239647836241]
[0.17595247987620477, 0.22441551209636112, 0.19330870572710512, 0.16944918055392122, 0.17701136029771736]
Average Feature Importances: [0.28205149 0.27015105 0.23488761 0.18376268 0.02914718]
Mean R^2 Score: 0.11913939515266636
